In [1]:
import os
import sys
import json
import tqdm
from dotenv import load_dotenv
load_dotenv()

project_root = os.path.abspath(os.path.join(os.path.dirname('../')))
sys.path.append(project_root)

import core.data_model as model
from core.database import Database
db = Database()

In [9]:
tag_id = '64829b1a2e33334d7f99c66d'
res = model.Gallery.objects(tags=tag_id)
print(res._query)

{'tags': ObjectId('64829b1a2e33334d7f99c66d')}


In [2]:
model_name='Ashley Adams'
model_id = "647c36ffa6f57b54a3e18159"
# galleries = model.Gallery.objects(channels__size__gte=2)


checked_channels = set()
galleries = model.Gallery.objects(__raw__={  '$expr': { '$gte': [{ '$size': '$channels' }, 2]} })
for gallery in tqdm.tqdm(galleries, desc="Update Channel's parent index"):
  channels = gallery.channels
  for i in range(1, len(channels)):
    if channels[i].id in checked_channels: continue
    if channels[i] not in channels[0].children:
      channels[0].children.append(channels[i])
      print("save new child")
      channels[0].save() 
    if channels[i].parent != channels[0]:
      channels[i].parent = channels[0]
      print("save new parent")
      channels[i].save()
    checked_channels.add(channels[i].id)  
del checked_channels
# for g in galleries:
#   print(g.id)
#   print(g.path)  
#   print(g.gid)
#   print(g.models)
#   print("------------------")
  
# model.Model.objects(id=model_id)

Update Channel's parent index:   7%|▋         | 30156/432447 [01:06<14:41, 456.14it/s]  


KeyboardInterrupt: 